In [ ]:
# Importing libraries

import os
import torch
import argparse
import numpy as np 
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import *
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

In [ ]:
# importing models
import torchvision.models as models

In [ ]:
transform_train = transforms.Compose([transforms.Resize((256,256)),
                                    transforms.RandomCrop((224,224)), # resizing image size to (224,224)
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
                                  ])

transform_val = transforms.Compose([transforms.Resize((224,224)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225)),
                                ])

In [ ]:
dir = 'cats_and_dogs_dataset' # datasets path

In [ ]:
trainset = ImageFolder(root = os.path.join(dir, 'train'), transform=transform_train)
valset = ImageFolder(root = os.path.join(dir, 'valid'), transform=transform_val)

In [ ]:
train_data

In [ ]:
# Loading images
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers = 2)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False, num_workers = 2)

In [ ]:
# Here i have used resnet101 and its last layer(output layer) should be modeified as per requirement
model = models.resnet101(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# freezing the models weights
for param in model.parameters():
  param.requires_grad = False

In [ ]:
# changing last layer (output layer) of the model
model.fc=nn.Linear(2048,2)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler=StepLR(optimizer,step_size=3)
criterion=nn.CrossEntropyLoss()
criterion.cuda()

CrossEntropyLoss()

In [ ]:
# training the model

def train(epoch):
	print('\nEpoch: %d' % epoch)
	scheduler.step()
	model.train()
	for batch_idx,(img,label) in enumerate(trainloader):
		image=Variable(img.cuda())
		label=Variable(label.cuda())
		optimizer.zero_grad()
		out=model(image)
		loss=criterion(out,label)
		loss.backward()
		optimizer.step()
		print("Epoch:%d [%d|%d] loss:%f" %(epoch,batch_idx,len(trainloader),loss.mean()))

def val(epoch):
	print("\nValidation Epoch: %d" %epoch)
	model.eval()
	total=0
	correct=0
	with torch.no_grad():
		for batch_idx,(img,label) in enumerate(valloader):
			image=Variable(img.cuda())
			label=Variable(label.cuda())
			out=model(image)
			_,predicted=torch.max(out.data,1)
			total+=image.size(0)
			correct+=predicted.data.eq(label.data).cpu().sum()
	print("Acc: %f "% ((1.0*correct.numpy())/total))

In [ ]:
epochs = 1
for epoch in range(epochs):
	train(epoch)
	val(epoch)
torch.save(model.state_dict(),'ckp/model.pth')


Epoch: 0


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch:0 [0|360] loss:0.681532
Epoch:0 [1|360] loss:0.714852
Epoch:0 [2|360] loss:0.683707
Epoch:0 [3|360] loss:0.685643
Epoch:0 [4|360] loss:0.638273
Epoch:0 [5|360] loss:0.611862
Epoch:0 [6|360] loss:0.546264
Epoch:0 [7|360] loss:0.505272
Epoch:0 [8|360] loss:0.472475
Epoch:0 [9|360] loss:0.458705
Epoch:0 [10|360] loss:0.451661
Epoch:0 [11|360] loss:0.424559
Epoch:0 [12|360] loss:0.352173
Epoch:0 [13|360] loss:0.356911
Epoch:0 [14|360] loss:0.361575
Epoch:0 [15|360] loss:0.282760
Epoch:0 [16|360] loss:0.283295
Epoch:0 [17|360] loss:0.251562
Epoch:0 [18|360] loss:0.209751
Epoch:0 [19|360] loss:0.237444
Epoch:0 [20|360] loss:0.211884
Epoch:0 [21|360] loss:0.218041
Epoch:0 [22|360] loss:0.196778
Epoch:0 [23|360] loss:0.335264
Epoch:0 [24|360] loss:0.179375
Epoch:0 [25|360] loss:0.183151
Epoch:0 [26|360] loss:0.173590
Epoch:0 [27|360] loss:0.145600
Epoch:0 [28|360] loss:0.144778
Epoch:0 [29|360] loss:0.124407
Epoch:0 [30|360] loss:0.176463
Epoch:0 [31|360] loss:0.117856
Epoch:0 [32|360] l

In [ ]:
# Saving the trained model
torch.save(model.state_dict(),'model.pth')

In [ ]:
# Loading the saved model

# model = Net()
model.load_state_dict(torch.load('model.pth'))
model.eval()

/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla T4 with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the Tesla T4 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 